# Set Up
In order to collect data from the Federal Reserve Economic Data (FRED), you need to create
an account in the website below, then go to the top right on My Account > API Keys and the on
the top left of the page, click Request API Key to get your API key.

Website: https://fredaccount.stlouisfed.org/apikeys

In [1]:
# In order to use the FRED API service, you need to install it in your coding environment
# as well as the pandas-datareader
# NOTE: you may need to restart the kernel in order to use this after installing
# Uncomment the lines below to install

# pip install fredapi
# !pip install pandas-datareader -q

# Collecting Data

Below we will import our dependencies and write a function that will retrieve the data we need for the analysis

In [3]:
# Import Dependencies
%matplotlib inline
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import numpy as np
from config import fredapi_key
import requests
import time
from datetime import datetime
import json

ImportError: cannot import name 'fredapi_key' from 'config' (/Users/lesliewerner/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/config/__init__.py)

In [3]:
def fred_data(series, start_date, end_date):
  df = pdr.DataReader(series, 'fred', start_date, end_date)
  return df.reset_index()

# Finding Housing Market Indicators

Knowing what data to collect is important. Luckily FRED provides a lot of data from CPI, Interest Rates, Unemployment rates, minimum wage, etc.

## FRED Series Data

Below you will see what datasets we are using and why, as well as the units and frequency. The abbreviation in the beginning of the bulletin is how FRED id's the dataset

### US Economic Data

- UNRATE - Unemployment Rate\
    Units: Percent, Seasonally Adjusted\
    Frequency: Monthly
    
- CPIAUCSL - Consumer Price Index for All Urban Consumers: All Items in U.S. City Average\
    Units: Index 1982-1984=100, Seasonally Adjusted\
    Frequency: Monthly

### Housing-Specific Data

- COMPUTSA - New Privately-Owned Housing Units Completed: Total Units\
    Units: Thousands of Units\
    Frequency: Monthly
    
- MORTGAGE30US - 30-Year Fixed Rate Mortgage Average in the United States\
    Units: Percent, Not Seasonally Adjusted\
    Frequency: Weekly, Ending Thursday
        
- CSUSHPISA - S&P/Case-Shiller U.S. National Home Price Index\
    Units: Index Jan 2000=100, Seasonally Adjusted\
    Frequency: Monthly

# Making API Call and Creating DataFrame

In [4]:
series = ['CPIAUCSL', 'UNRATE', 'COMPUTSA', 'CSUSHPISA']
start_date = '1991-01-01'
end_date = '2022-01-01'

housing_data_df = fred_data(series, start_date, end_date)
housing_data_df

,DATE,CPIAUCSL,UNRATE,COMPUTSA,CSUSHPISA
0,1991-01-01,134.700,6.4,1149.0,75.915
1,1991-02-01,134.800,6.6,1090.0,75.734
2,1991-03-01,134.800,6.8,1176.0,75.569
3,1991-04-01,135.100,6.7,1093.0,75.565
4,1991-05-01,135.600,6.9,1070.0,75.765
...,...,...,...,...,...
368,2021-09-01,274.214,4.7,1235.0,270.257
369,2021-10-01,276.590,4.6,1255.0,273.136
370,2021-11-01,278.524,4.2,1404.0,276.417
371,2021-12-01,280.126,3.9,1333.0,279.889


In [5]:
mortgage_rate_df = fred_data('MORTGAGE30US', start_date, '2022-02-01')
mortgage_rate_df

,DATE,MORTGAGE30US
0,1991-01-04,9.56
1,1991-01-11,9.63
2,1991-01-18,9.75
3,1991-01-25,9.61
4,1991-02-01,9.56
...,...,...
1617,2021-12-30,3.11
1618,2022-01-06,3.22
1619,2022-01-13,3.45
1620,2022-01-20,3.56


## Cleaning the 30-Year Fixed Mortgage Rate Column

All of our FRED series have a monthly frequency except for the 30-year mortgage rate, which is calculated weekly at the end of every Thursday. By using the code in the cell below, we aggregate the rows with similar month and year date and take the average, so the DataFrame below will be added to the housing_data_df using that DataFrame's index.

In [6]:
new_mortgage_df = mortgage_rate_df.groupby(pd.PeriodIndex(mortgage_rate_df['DATE'], freq="M"))['MORTGAGE30US'].mean().reset_index()
new_mortgage_df['DATE'] = housing_data_df['DATE']
new_mortgage_df

,DATE,MORTGAGE30US
0,1991-01-01,9.6375
1,1991-02-01,9.3650
2,1991-03-01,9.5000
3,1991-04-01,9.4925
4,1991-05-01,9.4720
...,...,...
368,2021-09-01,2.9000
369,2021-10-01,3.0675
370,2021-11-01,3.0675
371,2021-12-01,3.0980


In [7]:
housing_data_df['MORTGAGE30US'] = mortgage_rate_df['MORTGAGE30US']
housing_data_df

,DATE,CPIAUCSL,UNRATE,COMPUTSA,CSUSHPISA,MORTGAGE30US
0,1991-01-01,134.700,6.4,1149.0,75.915,9.56
1,1991-02-01,134.800,6.6,1090.0,75.734,9.63
2,1991-03-01,134.800,6.8,1176.0,75.569,9.75
3,1991-04-01,135.100,6.7,1093.0,75.565,9.61
4,1991-05-01,135.600,6.9,1070.0,75.765,9.56
...,...,...,...,...,...,...
368,2021-09-01,274.214,4.7,1235.0,270.257,6.99
369,2021-10-01,276.590,4.6,1255.0,273.136,7.12
370,2021-11-01,278.524,4.2,1404.0,276.417,7.03
371,2021-12-01,280.126,3.9,1333.0,279.889,7.06


In [8]:
housing_data_df.to_csv('data_csv/fred_housing_data.csv', index=False)